In [364]:
from collections import Counter
from itertools import chain

def count_symbols(image):
    pixels = image.getdata()
    values = chain.from_iterable(pixels)
    counts = Counter(values).items()
    return sorted(counts, key=lambda x:x[::-1])

In [478]:
def build_tree(counts) :
    nodes = [entry[::-1] for entry in counts] # Reverse each (symbol,count) tuple
#     print("no",nodes)
    while len(nodes) > 1 :
        leastTwo = tuple(nodes[0:2]) # get the 2 to combine
        theRest = nodes[2:] # all the others
        combFreq = leastTwo[0][0] + leastTwo[1][0]  # the branch points freq
        nodes = theRest + [(combFreq, leastTwo)] # add branch point to the end
        print("node")
#         nodes.sort(key=lambda a:a[1]) # sort it into place
    return nodes[0] # Return the single tree inside the list

In [479]:
def trim_tree(tree) :
    p = tree[1] # Ignore freq count in [0]
    if type(p) is tuple: # Node, trim left then right and recombine
        return (trim_tree(p[0]), trim_tree(p[1]))
    return p # Leaf, just return it

In [480]:
def assign_codes_impl(codes, node, pat):
    if type(node) == tuple:
        assign_codes_impl(codes, node[0], pat + [0]) # Branch point. Do the left branch
        assign_codes_impl(codes, node[1], pat + [1]) # then do the right branch.
    else:
        codes[node] = pat # A leaf. set its code

def assign_codes(tree):
    codes = {}
    assign_codes_impl(codes, tree, [])
    return codes


In [481]:
def to_binary_list(n):
    """Convert integer into a list of bits"""
    return [n] if (n <= 1) else to_binary_list(n >> 1) + [n & 1]

def from_binary_list(bits):
    """Convert list of bits into an integer"""
    result = 0
    for bit in bits:
        result = (result << 1) | bit
    return result

def pad_bits(bits, n):
    """Prefix list of bits with enough zeros to reach n digits"""
    assert(n >= len(bits))
    return ([0] * (n - len(bits)) + bits)

In [482]:
class OutputBitStream(object): 
    def __init__(self, file_name): 
        self.file_name = file_name
        self.file = open(self.file_name, 'wb') 
        self.bytes_written = 0
        self.buffer = []

    def write_bit(self, value):
        self.write_bits([value])

    def write_bits(self, values):
        self.buffer += values
        while len(self.buffer) >= 8:
            self._save_byte()        

    def flush(self):
        if len(self.buffer) > 0: # Add trailing zeros to complete a byte and write it
            self.buffer += [0] * (8 - len(self.buffer))
            self._save_byte()
        assert(len(self.buffer) == 0)

    def _save_byte(self):
        bits = self.buffer[:8]
        self.buffer[:] = self.buffer[8:]

        byte_value = from_binary_list(bits)
        self.file.write(bytes([byte_value]))
        self.bytes_written += 1

    def close(self): 
        self.flush()
        self.file.close()

In [483]:
class InputBitStream(object): 
    def __init__(self, file_name): 
        self.file_name = file_name
        self.file = open(self.file_name, 'rb') 
        self.bytes_read = 0
        self.buffer = []

    def read_bit(self):
        return self.read_bits(1)[0]

    def read_bits(self, count):
        while len(self.buffer) < count:
            self._load_byte()
        result = self.buffer[:count]
        self.buffer[:] = self.buffer[count:]
        return result

    def flush(self):
        assert(not any(self.buffer))
        self.buffer[:] = []

    def _load_byte(self):
        value = ord(self.file.read(1))
        self.buffer += pad_bits(to_binary_list(value), 8)
        self.bytes_read += 1

    def close(self): 
        self.file.close()


In [484]:
from PIL import Image

def compressed_size(counts, codes):
    header_size = 2 * 16 # height and width as 16 bit values

    tree_size = len(counts) * (1 + 8) # Leafs: 1 bit flag, 8 bit symbol each
    tree_size += len(counts) - 1 # Nodes: 1 bit flag each
    if tree_size % 8 > 0: # Padding to next full byte
        tree_size += 8 - (tree_size % 8)

    # Sum for each symbol of count * code length
    pixels_size = sum([count * len(codes[symbol]) for symbol, count in counts])
    if pixels_size % 8 > 0: # Padding to next full byte
        pixels_size += 8 - (pixels_size % 8)

    return (header_size + tree_size + pixels_size) / 8

def encode_header(image, bitstream):
    height_bits = pad_bits(to_binary_list(image.height), 16)
    bitstream.write_bits(height_bits)    
    width_bits = pad_bits(to_binary_list(image.width), 16)
    bitstream.write_bits(width_bits)

def encode_tree(tree, bitstream):
    if type(tree) == tuple: # Note - write 0 and encode children
        bitstream.write_bit(0)
        encode_tree(tree[0], bitstream)
        encode_tree(tree[1], bitstream)
    else: # Leaf - write 1, followed by 8 bit symbol
        bitstream.write_bit(1)
        symbol_bits = pad_bits(to_binary_list(tree), 8)
        bitstream.write_bits(symbol_bits)

def encode_pixels(image, codes, bitstream):
    for pixel in image.getdata():
        for value in pixel:
            bitstream.write_bits(codes[value])

def compress_image(in_file_name, out_file_name):
    print('Compressing "%s" -> "%s"' % (in_file_name, out_file_name))
    image = Image.open(in_file_name)
    print('Image shape: (height=%d, width=%d)' % (image.height, image.width))
    size_raw = raw_size(image.height, image.width)
    print('RAW image size: %d bytes' % size_raw)
    counts = count_symbols(image)
    print('Counts: %s' % counts)
    tree = build_tree(counts)
    print('Tree: %s' % str(tree))
    trimmed_tree = trim_tree(tree)
    print('Trimmed tree: %s' % str(trimmed_tree))
    codes = assign_codes(trimmed_tree)
    print('Codes: %s' % codes)

    size_estimate = compressed_size(counts, codes)
    print('Estimated size: %d bytes' % size_estimate)

    print('Writing...')
    stream = OutputBitStream(out_file_name)
    print('* Header offset: %d' % stream.bytes_written)
    encode_header(image, stream)
    stream.flush() # Ensure next chunk is byte-aligned
    print('* Tree offset: %d' % stream.bytes_written)
    encode_tree(trimmed_tree, stream)
    stream.flush() # Ensure next chunk is byte-aligned
    print('* Pixel offset: %d' % stream.bytes_written)
    encode_pixels(image, codes, stream)
    stream.close()

    size_real = stream.bytes_written
    print('Wrote %d bytes.' % size_real)

    print('Estimate is %scorrect.' % ('' if size_estimate == size_real else 'in'))
    print('Compression ratio: %0.2f' % (float(size_raw) / size_real))

In [485]:
# Decompression

In [486]:
from PIL import Image

def decode_header(bitstream):
    height = from_binary_list(bitstream.read_bits(16))
    width = from_binary_list(bitstream.read_bits(16))
    return (height, width)

# https://stackoverflow.com/a/759766/3962537
def decode_tree(bitstream):
    flag = bitstream.read_bits(1)[0]
    if flag == 1: # Leaf, read and return symbol
        return from_binary_list(bitstream.read_bits(8))
    left = decode_tree(bitstream)
    right = decode_tree(bitstream)
    return (left, right)

def decode_value(tree, bitstream):
    bit = bitstream.read_bits(1)[0]
    node = tree[bit]
    if type(node) == tuple:
        return decode_value(node, bitstream)
    return node

def decode_pixels(height, width, tree, bitstream):
    pixels = bytearray()
    for i in range(height * width * 3):
        pixels.append(decode_value(tree, bitstream))
    return Image.frombytes('RGB', (width, height), bytes(pixels))

def decompress_image(in_file_name, out_file_name):
    print('Decompressing "%s" -> "%s"' % (in_file_name, out_file_name))

    print('Reading...')
    stream = InputBitStream(in_file_name)
    print('* Header offset: %d' % stream.bytes_read)
    height, width = decode_header(stream)
    stream.flush() # Ensure next chunk is byte-aligned
    print('* Tree offset: %d' % stream.bytes_read)    
    trimmed_tree = decode_tree(stream)
    stream.flush() # Ensure next chunk is byte-aligned
    print('* Pixel offset: %d' % stream.bytes_read)
    image = decode_pixels(height, width, trimmed_tree, stream)
    stream.close()
    print('Read %d bytes.' % stream.bytes_read)

    print('Image size: (height=%d, width=%d)' % (height, width))
    print('Trimmed tree: %s' % str(trimmed_tree))
    image.save(out_file_name)

In [487]:
# Test Run

In [488]:
from PIL import ImageChops
import time

def raw_size(width, height):
    header_size = 2 * 16 # height and width as 16 bit values
    pixels_size = 3 * 8 * width * height # 3 channels, 8 bits per channel
    return (header_size + pixels_size) / 8

def images_equal(file_name_a, file_name_b):
    image_a = Image.open(file_name_a)
    image_b = Image.open(file_name_b)

    diff = ImageChops.difference(image_a, image_b)

    return diff.getbbox() is None

if __name__ == '__main__':
    start = time.time()

    compress_image('img.jpg', 'answer.txt')

    print('-' * 40)

    decompress_image('answer.txt', 'img_out.jpg')

    stop = time.time()
    times = (stop - start) * 1000

    print('-' * 40)

    print('Run time takes %d miliseconds' % times)
    print('Images equal = %s' % images_equal('img.jpg', 'img_out.jpg'))

Compressing "img.jpg" -> "answer.txt"
Image shape: (height=240, width=160)
RAW image size: 115204 bytes
Counts: [(241, 1), (243, 1), (245, 1), (246, 1), (248, 1), (250, 1), (252, 1), (236, 2), (239, 2), (240, 2), (242, 2), (237, 4), (244, 4), (235, 5), (238, 5), (233, 6), (234, 6), (231, 7), (232, 8), (229, 9), (230, 9), (255, 11), (226, 13), (228, 15), (225, 22), (227, 25), (221, 30), (219, 39), (222, 39), (223, 39), (220, 40), (224, 40), (218, 46), (213, 48), (217, 48), (215, 50), (209, 51), (211, 52), (207, 56), (214, 56), (204, 57), (216, 59), (210, 60), (212, 62), (201, 63), (195, 65), (208, 67), (199, 70), (205, 72), (196, 75), (203, 76), (202, 77), (192, 78), (200, 78), (189, 79), (197, 80), (191, 81), (193, 81), (190, 83), (194, 83), (188, 89), (198, 89), (206, 90), (187, 107), (186, 118), (185, 119), (184, 139), (183, 155), (182, 161), (180, 162), (181, 163), (179, 212), (178, 240), (82, 248), (88, 255), (96, 256), (90, 262), (95, 264), (87, 268), (92, 271), (174, 272), (86, 2

Wrote 114714 bytes.
Estimate is correct.
Compression ratio: 1.00
----------------------------------------
Decompressing "answer.txt" -> "img_out.jpg"
Reading...
* Header offset: 0
* Tree offset: 4
* Pixel offset: 318
Read 114714 bytes.
Image size: (height=240, width=160)
Trimmed tree: (((((((115, 116), (112, 117)), ((0, (241, 243)), ((245, 246), (248, 250)))), ((((252, 236), (239, 240)), ((242, 237), (244, 235))), (((238, 233), (234, 231)), ((232, 229), (230, 255))))), (((((226, 228), (225, 227)), ((221, 219), (222, 223))), (((220, 224), (218, 213)), ((217, 215), (209, 211)))), ((((207, 214), (204, 216)), ((210, 212), (201, 195))), (((208, 199), (205, 196)), ((203, 202), (192, 200)))))), ((((((189, 197), (191, 193)), ((190, 194), (188, 198))), (((206, 187), (186, 185)), ((184, 183), (182, 180)))), ((((181, 179), (178, 82)), ((88, 96), (90, 95))), (((87, 92), (174, 86)), ((98, 177), (85, 176))))), (((((93, 80), (89, 76)), ((175, 70), (97, 94))), (((91, 75), (83, 74)), ((78, 79), (84, 81

In [489]:
a = [(533, 2), (615, 3), (732, 4), (709, 5), (659, 6), (719, 7), (747, 8), (743, 9), (797, 10), (790, 11), (772, 12), (777, 13), (783, 14), (906, 15), (955, 16), (980, 17), (869, 18), (850, 19), (837, 20), (819, 21), (786, 22), (840, 23), (794, 24), (784, 25), (778, 26), (768, 27), (779, 28), (801, 29), (823, 30), (803, 31), (732, 32), (708, 33), (716, 34), (677, 35), (681, 36), (661, 37), (658, 38), (618, 39), (661, 40), (605, 41), (593, 42), (565, 43), (591, 44), (551, 45), (535, 46), (549, 47), (541, 48), (507, 49), (489, 50), (521, 51), (508, 52), (523, 53), (465, 54), (459, 55), (454, 56), (467, 57), (439, 58), (451, 59), (407, 60), (433, 61), (390, 62), (385, 63), (370, 64), (346, 65), (395, 66), (420, 67), (365, 68), (321, 69), (295, 70), (334, 71), (332, 72), (354, 73), (307, 74), (300, 75), (294, 76), (319, 77), (307, 78), (308, 79), (291, 80), (315, 81), (248, 82), (305, 83), (312, 84), (284, 85), (273, 86), (268, 87), (255, 88), (291, 89), (262, 90), (298, 91), (271, 92), (286, 93), (297, 94), (264, 95), (256, 96), (296, 97), (279, 98), (320, 99), (453, 100), (542, 101), (589, 102), (648, 103), (607, 104), (634, 105), (709, 106), (731, 107), (783, 108), (792, 109), (903, 110), (1039, 111), (1133, 112), (1073, 113), (1047, 114), (1103, 115), (1126, 116), (1134, 117), (997, 118), (991, 119), (949, 120), (914, 121), (820, 122), (852, 123), (934, 124), (839, 125), (822, 126), (793, 127), (788, 128), (810, 129), (789, 130), (780, 131), (803, 132), (893, 133), (779, 134), (859, 135), (805, 136), (761, 137), (720, 138), (757, 139), (658, 140), (617, 141), (648, 142), (650, 143), (608, 144), (647, 145), (597, 146), (644, 147), (642, 148), (647, 149), (665, 150), (778, 151), (853, 152), (760, 153), (678, 154), (648, 155), (667, 156), (629, 157), (625, 158), (567, 159), (571, 160), (580, 161), (661, 162), (771, 163), (966, 164), (883, 165), (758, 166), (575, 167), (620, 168), (526, 169), (527, 170), (468, 171), (402, 172), (361, 173), (272, 174), (294, 175), (284, 176), (282, 177), (240, 178), (212, 179), (162, 180), (163, 181), (161, 182), (155, 183), (139, 184), (119, 185), (118, 186), (107, 187), (89, 188), (79, 189), (83, 190), (81, 191), (78, 192), (81, 193), (83, 194), (65, 195), (75, 196), (80, 197), (89, 198), (70, 199), (78, 200), (63, 201), (77, 202), (76, 203), (57, 204), (72, 205), (90, 206), (56, 207), (67, 208), (51, 209), (60, 210), (52, 211), (62, 212), (48, 213), (56, 214), (50, 215), (59, 216), (48, 217), (46, 218), (39, 219), (40, 220), (30, 221), (39, 222), (39, 223), (40, 224), (22, 225), (13, 226), (25, 227), (15, 228), (9, 229), (9, 230), (7, 231), (8, 232), (6, 233), (6, 234), (5, 235), (2, 236), (4, 237), (5, 238), (2, 239), (2, 240), (2, 242), (4, 244), (1, 245), (1, 246), (1, 248), (1, 250), (1, 252), (11, 255), (2, ((1, 241), (1, 243)))]